## Naivni Bayesov klasifikator

In [1]:
class NaiveBayes:  
    """
    Naive Bayes classifier.
    
    :attribute self.probabilities
        Dictionary that stores
            - prior class probabilities P(Y)
            - attribute probabilities conditional on class P(X|Y)
    
    :attribute self.class_values
        All possible values of the class.
        
    :attribute self.variables
        Variables in the data. 
    
    :attribute self.trained
        Set to True after fit is called.
    """
    
    def __init__(self):
        self.trained       = False
        self.probabilities = dict()   
    
    
    def fit(self, data):
        """
        Fit a NaiveBayes classifier.
        
        :param data
            pandas dataframe.        
        """
        class_variable      = data.columns[-1]    # class variable (Y) 
        self.class_values   = pd.unique(data['survived'])    # possible class values
        self.variables      = data.columns[:-1]    # all other variables (X)
        
        n = len(data) # number of all data points
        
        # Compute P(Y)
        for y in self.class_values:

            # Compute class probabilities and correctly fill
            #   probabilities[y] = ... 
            # Select all examples (rows) with class = y
            data_subset = data.loc[data[class_variable] == y]
            m = len(data_subset)
        
            self.probabilities[y] = m/n
        
        # Compute P(X|Y)
        for y in self.class_values:
            
            # Select all examples (rows) with class = y
            data_subset = data.loc[data[class_variable] == y]
            p = len(data_subset)
            
            for variable in self.variables:
                for x in pd.unique(data[variable]):
                    
                    # Compute correct conditional class probability
                    #   probabilities[x, value, c] = ... 
                    # 
                    # Select all examples with class == y AND 
                    # variable x == value
                    # Hint: use SameValue filter twice
                    data_subset = data.loc[(data[variable] == x) & (data[class_variable] == y)]
                    m = len(data_subset)
                    
                    self.probabilities[variable, x, y] = m/p
    
        self.trained = True
           
    def predict_instance(self, row):
        """
        Predict a class value for one row.
        
        :param row
            Orange data Instance.
        :return 
            Class prediction.
        """
        curr_p = float("-inf")   # Current highest "probability" (unnormalized)
        curr_c = None            # Current most probable class
        
        for y in self.class_values:
            p = np.log(self.probabilities[y])
            for x in self.variables:
                p = p + np.log(self.probabilities[x, row[x], y])
            
            if p > curr_p:
                curr_p = p
                curr_c = y
                
        return curr_c, curr_p
        
    def predict(self, data):
        """
        Predict class labels for all rows in data.
        
        :param data
            Orange data Table.       
        :return y
            NumPy vector with predicted classes.
        """
        
        n = len(data)
        predictions = list()
        confidences = np.zeros((n, ))
        
        for i in range(len(data)):
            pred, cf = self.predict_instance(data.iloc[i])
            predictions.append(pred)
            confidences[i] = cf
    
        return predictions, confidences